In [1]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [4]:
from glob import glob
from tqdm import tqdm
import json
import os
import numpy as np
from transformers import AutoTokenizer
from streaming import MDSWriter
from tqdm import tqdm
import msgspec

In [5]:
from streaming.base.format.mds.encodings import Encoding, _encodings

class UInt16(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint16)

_encodings['uint16'] = UInt16

In [6]:
columns = {
    'input_ids': 'uint16',
}
compression = 'zstd'
hashes = 'sha1', 'xxh64'

In [7]:
split_by = 5000000

In [8]:
!mkdir partitions-filtered

In [9]:
index = 0
count = 0
a = open(f'partitions-filtered/combined-lm-{index}.jsonl', 'w')

with open('prepare-dedup-text-dataset-filtered.jsonl') as fopen:
    for l in tqdm(fopen):
        a.write(l)
        a.flush()
        count += 1
        if count >= split_by:
            a.close()
            index += 1
            count = 0
            a = open(f'partitions-filtered/combined-lm-{index}.jsonl', 'w')
            
a.close()

179327699it [23:33, 126847.98it/s]


In [10]:
block_size = 4096

def read_dataset(train_file, block_size = block_size):
    
    tokenizer = AutoTokenizer.from_pretrained(
        'malaysia-ai/bpe-tokenizer',
    )
    tokenizer.add_bos_token = False
    tokenizer.add_eos_token = False
    text_column_name = 'text'
    temp = []
    with open(train_file) as fopen:
        for l in fopen:
            l = msgspec.json.decode(l)
            tokenized = tokenizer(l['text'])['input_ids']
            temp.extend(tokenized)
            while len(temp) >= block_size:
                block = temp[:block_size]
                temp = temp[block_size:]
                if len(block) == block_size:
                    yield np.array(block).astype(np.uint16)

In [11]:
files = sorted(glob('partitions-filtered/combined-lm-*.jsonl'), key = lambda x: int(x.split('-')[-1].replace('.jsonl', '')))
files

['partitions-filtered/combined-lm-0.jsonl',
 'partitions-filtered/combined-lm-1.jsonl',
 'partitions-filtered/combined-lm-2.jsonl',
 'partitions-filtered/combined-lm-3.jsonl',
 'partitions-filtered/combined-lm-4.jsonl',
 'partitions-filtered/combined-lm-5.jsonl',
 'partitions-filtered/combined-lm-6.jsonl',
 'partitions-filtered/combined-lm-7.jsonl',
 'partitions-filtered/combined-lm-8.jsonl',
 'partitions-filtered/combined-lm-9.jsonl',
 'partitions-filtered/combined-lm-10.jsonl',
 'partitions-filtered/combined-lm-11.jsonl',
 'partitions-filtered/combined-lm-12.jsonl',
 'partitions-filtered/combined-lm-13.jsonl',
 'partitions-filtered/combined-lm-14.jsonl',
 'partitions-filtered/combined-lm-15.jsonl',
 'partitions-filtered/combined-lm-16.jsonl',
 'partitions-filtered/combined-lm-17.jsonl',
 'partitions-filtered/combined-lm-18.jsonl',
 'partitions-filtered/combined-lm-19.jsonl',
 'partitions-filtered/combined-lm-20.jsonl',
 'partitions-filtered/combined-lm-21.jsonl',
 'partitions-filtere

In [12]:
next(read_dataset(files[1]))

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

array([    1,   224, 22783, ..., 17580,  1156,   235], dtype=uint16)

In [13]:
!rm -rf tokenized_indexes-filtered
!mkdir tokenized_indexes-filtered

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
def loop(files):
    files, index = files
    out_root = f'tokenized_indexes-filtered/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    with MDSWriter(out=out_root, columns=columns, compression=compression, hashes=hashes) as out:
        for f in files:
            for block in tqdm(read_dataset(train_file = f)):
                sample = {
                    'input_ids': block
                }
                out.write(sample)

In [15]:
import mp
mp.multiprocessing(files, loop, cores = 5, returned = False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [18]:
from streaming import StreamingDataset

total = 0
for f in glob('tokenized_indexes-filtered/tokenized-*'):
    dataset = StreamingDataset(local = f)
    total += len(dataset)
total * block_size

18604802048